# Advent of Code 2023

## Contents
- [Day 1](#day-1)
- [Day 2](#day-2)
- [Day 3](#day-3)
- [Day 4](#day-4)
- [Day 5](#day-5)
- [Day 6](#day-6)
- [Day 7](#day-7)
- [Day 8](#day-8)
- [Day 9](#day-9)
- [Day 10](#day-10)
- [Day 11](#day-11)
- [Day 12](#day-12)

## Boilerplate

In [173]:
# SETUP #
import math
import operator
from functools import reduce

# TEST #
def test(test, solution):
    if test != solution:
        print(f"Test failed. Expected {solution}, but got {test}.")
    else: print("Test success!")

In [186]:
# FILE READING - Run *after* day config #
with open(f"inputs/{DAY}.txt", mode="rt") as f:
    PUZZLE_INPUT = f.read()

with open(f"test_inputs/{DAY}.txt", mode="rt") as f:
    TEST_INPUT = f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'inputs/7.txt'

## Day 1

[Link to puzzle](https://adventofcode.com/2023/day/1)

In [101]:
# DAY CONFIG #
DAY = "1"
TEST_SOLUTION_PART_1 = 142
TEST_SOLUTION_PART_2 = 281

In [104]:
%%time
# DAY 1 #
def run(part,i):
    
    # Part 2 only
    if part == 2:
        number_names = [
            ("one", "one1one"),
            ("two", "two2two"),
            ("three", "three3three"),
            ("four", "four4four"),
            ("five", "five5five"),
            ("six", "six6six"),
            ("seven", "seven7seven"),
            ("eight", "eight8eight"),
            ("nine", "nine9nine"),
        ]
        for pair in number_names:
            i = i.replace(pair[0], pair[1])

    total = 0
    # For each line, combine the first and last digits and add to total
    for line in i.splitlines():
        digits = ''.join(c for c in line if c.isdigit())
        total += int(digits[0] + digits[-1])
    return total

# Run test
test(run(1,"1abc2\npqr3stu8vwx\na1b2c3d4e5f\ntreb7uchet"), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 54951
Part 2: 55218
CPU times: user 7.99 ms, sys: 771 µs, total: 8.76 ms
Wall time: 8.25 ms


## Day 2

[Link to puzzle](https://adventofcode.com/2023/day/2)

In [105]:
# DAY CONFIG #
DAY = "2"
TEST_SOLUTION_PART_1 = 8
TEST_SOLUTION_PART_2 = 2286

In [108]:
%%time
# DAY 2 #
def run(part,i):
    
    def clean_input(i):
        # Parse the input into the following format:
        # {'1': {'red': 4, 'green': 2, 'blue': 6}, '2': {'red': 1, 'green': 3, 'blue': 4}...
        clean = {}

        # Input line format -> Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
        for line in i.splitlines():
            # Break into "Game X" and game data (list of hands), then isolate game ID
            id, game = line.split(": ")
            id = id.split(" ")[1]
            
            clean[id] = {"red": 0, "green": 0, "blue": 0}

            for round in game.split("; "):
                for r in round.split(", "):
                    amount, colour = r.split(" ")
                    # Update the value for a colour only if it is higher than the existing value
                    clean[id][colour] = max(clean[id][colour], int(amount))

        return clean

    def validate(id,game):
        # Return game ID for valid games, 0 for invalid
        max_values = {"red": 12, "green": 13, "blue": 14}

        for colour, value in game.items():
            if value > max_values[colour]:
                return 0
            
        return int(id)

    def get_power(game):
        return math.prod(game.values())

    data = clean_input(i)
    
    total = 0
    total_power = 0

    for id, game in data.items():
        total += validate(id, game)
        total_power += get_power(game)

    return total if (part == 1) else total_power

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 2449
Part 2: 63981
CPU times: user 3.75 ms, sys: 101 µs, total: 3.86 ms
Wall time: 3.94 ms


## Day 3

[Link to puzzle](https://adventofcode.com/2023/day/3)

In [109]:
# DAY CONFIG #
DAY = "3"
TEST_SOLUTION_PART_1 = 4361
TEST_SOLUTION_PART_2 = 467835

In [111]:
%%time
# DAY 3 #
def run(part,i):
    # Convert input to 2D matrix
    i = list(map(list, i.splitlines()))

    checked = [[] for _ in i]
    parts = []
    gears = []

    def find_adjacent_numbers(row, column) -> ([int], int):
        adjacent_count = 0
        numbers = []
        gear_power = 0

        # Check row and column before and after
        for r in range(row-1, row+2):
            for c in range(column-1, column+2):
                if number := check_digit(r,c): 
                    numbers.append(number)
                    adjacent_count += 1

        if i[row][column] == '*' and adjacent_count == 2:
            gear_power = math.prod(numbers)
        
        return (numbers, gear_power)

    def check_digit(row, column) -> int:
        if row < 0 or row >= len(i) or column < 0 or column >= len(i[row]):
            return # out of bounds
        
        if i[row][column].isdigit() and column not in checked[row]:
            # Walk back to start of digit and save it
            number = ''
            while column > 0 and i[row][column-1].isdigit():
                column -= 1
                
            while column < len(i[row]) and i[row][column].isdigit():
                number += i[row][column]
                checked[row].append(column)
                column += 1

            return int(number)

    for row in range(len(i)):
        for column in range(len(i[row])):
            # Check for symbol, find numbers near it
            if not i[row][column].isdigit() and i[row][column] != '.':
                numbers, score = find_adjacent_numbers(row, column)
                parts.extend(numbers)
                gears.append(score)

    return sum(parts) if part == 1 else sum(gears)

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 556057
Part 2: 82824352
CPU times: user 17.2 ms, sys: 1.43 ms, total: 18.7 ms
Wall time: 17.6 ms


## Day 4

[Link to puzzle](https://adventofcode.com/2023/day/4)

In [112]:
# DAY CONFIG #
DAY = "4"
TEST_SOLUTION_PART_1 = 13
TEST_SOLUTION_PART_2 = 30

In [115]:
%%time
# DAY 4 #

def run(part,i):
    
    def clean_input(i):
        # Format -> Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
        # Return generator of id/wins tuples
        for line in i.splitlines():
            id, data = line.split(": ")
            id = int(id.split(" ")[-1])
            winners, card = (set(x.split()) for x in data.split(" | "))
            wins = len(winners & card)
            yield id, wins
    
    score = 0
    copies = [1] * len(i.splitlines()) # 1 copy of each card to start

    for id, wins in clean_input(i):
        score += 1 << (wins - 1) if wins > 0 else 0 # bit shift - same as 2^(wins-1)
        for x in range(wins):
            copies[id+x] += copies[id-1]

    return score if part == 1 else sum(copies)

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 28750
Part 2: 10212704
CPU times: user 6.17 ms, sys: 535 µs, total: 6.71 ms
Wall time: 6.52 ms


## Day 5

[Link to puzzle](https://adventofcode.com/2023/day/5)

In [116]:
# DAY CONFIG #
DAY = "5"
TEST_SOLUTION_PART_1 = 35
TEST_SOLUTION_PART_2 = 46

In [119]:
%%time
# DAY 5 #

def run(part,i):

    def clean_input(input):
        groups = input.split("\n\n")
        seeds = groups[0].split()[1:]
 
        mappings = []
        for map in groups[1:]:
            section = []
            for line in map.splitlines()[1:]:
                start_dest, start_source, length = [int(val) for val in line.split()]
                section.append((range(start_source, start_source + length), start_dest))
            mappings.append(section)

        return seeds, mappings

    def map_value(value, section):
        for range, destination in section:
            if value in range:
                return destination + (value - range.start)
        return value

    def check_ranges(ranges_to_check, mapping):
        new_ranges = []

        for current_range in ranges_to_check:
            for map_range, next_start in mapping:
                intersection = range(max(map_range.start, current_range.start), min(map_range.stop, current_range.stop))
                
                if intersection: # if there is a valid intersection
                    offset = next_start - map_range.start
                    new_ranges.append(range(intersection.start + offset, intersection.stop + offset))

                    # Update ranges to check based on the intersection
                    if intersection.start > current_range.start:
                        ranges_to_check.append(range(current_range.start, intersection.start))

                    if intersection.stop < current_range.stop:
                        ranges_to_check.append(range(intersection.stop, current_range.stop))

                    break
            else:
                new_ranges.append(current_range)

        return new_ranges
    
    # Main functions #
    
    def part1(): 
        locations = []   
        seeds, mappings = clean_input(i)
        for seed in seeds:
            for section in mappings:
                seed = map_value(int(seed), section)
            locations.append(seed)   
        return min(locations)

    def part2():
        seeds, mappings = clean_input(i)
        seed_ranges = [
            range(int(seeds[i]), int(seeds[i + 1]) + int(seeds[i])) 
            for i in range(0, len(seeds), 2)
        ]

        lowest = float('inf') # really big number
        for sr in seed_ranges:
            ranges = [sr]

            for mapping in mappings:
                ranges = check_ranges(ranges, mapping)

            lowest = min(lowest, min(r.start for r in ranges))

        return lowest
                
    return part1() if part == 1 else part2()

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 313045984
Part 2: 20283860
CPU times: user 5.69 ms, sys: 549 µs, total: 6.23 ms
Wall time: 5.85 ms


## Day 6

[Link to puzzle](https://adventofcode.com/2023/day/6)

In [137]:
# DAY CONFIG #
DAY = "6"
TEST_SOLUTION_PART_1 = 288
TEST_SOLUTION_PART_2 = 71503

In [184]:
%%time
# DAY 6 #

def run(part,i):
    
    def clean_input(input, combined=True):
        lines = input.splitlines()
        if combined:
            # Tuple of (time, distance)
            result = (int(''.join(line.split()[1:])) for line in lines)
        else:
            # List of (time, distance) for each race
            result = list(zip(map(int, lines[0].split()[1:]), map(int, lines[1].split()[1:])))
        return result

    def winning_cases_brute(time, distance):
        # Simple solution, ~5s for part 2
        # True/False for win/lose
        wins = [speed * (time - speed) > distance for speed in range(time)]
        return sum(wins)

    def winning_cases_quadratic(time, distance):
        # Quadratic equation time!
        # a = 1, b = -time, c = distance
        sqrt_discriminant = math.sqrt(time**2 - 4*distance)
        upper, lower = (time + sqrt_discriminant) / 2, (time - sqrt_discriminant) / 2
        return math.ceil(upper) - math.floor(lower) -1
    
    # Main functions #

    def part1():
        # Multiply the winning cases for each race
        races = clean_input(i, False)
        return reduce(operator.mul, (winning_cases_quadratic(*race) for race in races), 1)

    def part2():
        # Get number of winning cases for single race
        time, distance = clean_input(i)
        return winning_cases_quadratic(time, distance)
                
    return part1() if part == 1 else part2()

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 252000
Part 2: 36992486
CPU times: user 403 µs, sys: 66 µs, total: 469 µs
Wall time: 449 µs


## Day 7

[Link to puzzle](https://adventofcode.com/2023/day/7)

In [185]:
# DAY CONFIG #
DAY = "7"
TEST_SOLUTION_PART_1 = 1
TEST_SOLUTION_PART_2 = 1

In [187]:
%%time
# DAY 7 #

def run(part,i):
    
    def clean_input(input):
        return 0
    
    # Main functions #

    def part1():
        return 0

    def part2():
        return 0
                
    return part1() if part == 1 else part2()

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test failed. Expected 1, but got 0.
Test failed. Expected 1, but got 0.
Part 1: 0
Part 2: 0
CPU times: user 123 µs, sys: 16 µs, total: 139 µs
Wall time: 128 µs
